## `rocky`

### import rocky code

In [1]:
import sys
mydir = './src'
sys.path.append(mydir)
from rocky.core import rocky
from rocky.triangle import Triangle

import pandas as pd
import numpy as np
import plotly.express as px

### old way to make a model:

In [2]:
# Create a Rocky object
rky = rocky()

# add triangle
rocky_inputs_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\rocky-inputs.xlsx"
rky.FromExcel(filename=rocky_inputs_file, sheet_name='triangle', id='paid_loss')
# rky.SampleTri('taylor_ashe')

# add model
rky.AddModel('icrfs', model_class='loglinear', tri=rky.paid_loss, use_cal=True, plot_width=None)

# set hyperparameters
rky.icrfs.SetHyperparameters(alpha=0.05, l1_ratio=0.1)

# fit model
rky.icrfs.Fit()

# basic hetero adjustment (one-step, no cross-validation,
# just one parameter per development period)
rky.icrfs.FitHetero()

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.
Fitting hetero adjustment: (Step/RMSE/L2-Norm)
1/0.0987 2/0.0178 3/0.0030 

In [3]:
rky.icrfs.TrendPlot()

In [4]:
rky.icrfs.FitPlot('dev')

In [15]:
rky.icrfs.ResidualPlot('a', color_col='cal')

In [5]:
rky.icrfs.ResidualPlot(color_col='yhat')

In [6]:
rky.icrfs.Combine('accident', [1994,1995], 'acc_1994_1995')

In [7]:
rky.icrfs.GetX('train')

,intercept,accident_period_1987,accident_period_1988,accident_period_1989,accident_period_1990,accident_period_1991,accident_period_1992,accident_period_1993,accident_period_1996,accident_period_1997,...,calendar_period_067,calendar_period_068,calendar_period_069,calendar_period_070,calendar_period_071,calendar_period_072,calendar_period_073,calendar_period_074,calendar_period_075,acc_1994_1995
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
950,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
953,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
989,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
rky.icrfs.GetParameters()

,names,param,param_type
0,intercept,1.963017,intercept
1,accident_period_1987,0.268536,accident
2,accident_period_1988,-0.064878,accident
3,accident_period_1989,-0.099071,accident
4,accident_period_1990,-0.161787,accident
...,...,...,...
143,calendar_period_072,0.000000,calendar
144,calendar_period_073,0.000000,calendar
145,calendar_period_074,0.000000,calendar
146,calendar_period_075,0.000000,calendar


In [9]:
rky.icrfs.TrendPlot()

In [10]:
rky.icrfs.Ultimate().tail(10)

accident_period
2014-01-01     784803.0
2015-01-01     970355.0
2016-01-01     949414.0
2017-01-01     983747.0
2018-01-01    1125556.0
2019-01-01    1002553.0
2020-01-01     923018.0
2021-01-01     943817.0
2022-01-01     991138.0
2023-01-01    1030707.0
Name: Log-linear Ultimate, dtype: float64

In [11]:
rky.icrfs.Reserve().tail(10)

accident_period
2014-01-01     638546.9
2015-01-01     809779.7
2016-01-01     796490.6
2017-01-01     827154.0
2018-01-01     954211.7
2019-01-01     862652.0
2020-01-01     824061.4
2021-01-01     869863.0
2022-01-01     945041.8
2023-01-01    1011578.8
dtype: float64

### import rocky model config file

In [12]:
rocky_inputs_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\rocky-inputs.xlsx"
rocky_config_file = r"O:\PARM\Individual folders\A Weaver\Reserving\rocky\rockyConfigTemplate.py"

In [13]:
rky_config = rocky.load_config(rocky_config_file)

AttributeError: type object 'rocky' has no attribute 'load_config'

c:\Users\aweaver\rocky\rocky-app\./src\rocky\triangle.py:1915: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.X_base['is_observed'] = dm_total['is_observed'].astype(int)
c:\Users\aweaver\rocky\rocky-app\./src\rocky\triangle.py:1916: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.X_base['intercept'] = 1


rocky(id='rocky0', mod=(), f=(), plot=(), t="loss")

In [ ]:
rky.loss.df

development_period,12,24,36,48,60,72,84,96,108,120,...,348,360,372,384,396,408,420,432,444,456
accident_period,,,,,,,,,,,,,,,,,,,,,
1986-01-01,3321.0,6463.0,9864.0,13442.0,15589.0,16280.0,17706.0,17871.0,18060.0,18276.0,...,19153.0,19153.0,19153.0,19153.0,19153.0,19153.0,19153.0,19153.0,19153.0,19153.0
1987-01-01,3464.0,7249.0,10432.0,13469.0,15585.0,17202.0,18255.0,19194.0,19408.0,19456.0,...,20441.0,20441.0,20441.0,20441.0,20441.0,20441.0,20441.0,20441.0,20441.0,NaN
1988-01-01,3548.0,8346.0,11615.0,14230.0,17171.0,19355.0,19790.0,20547.0,20852.0,20891.0,...,21004.0,21004.0,21004.0,21004.0,21004.0,21004.0,21004.0,21004.0,NaN,NaN
1989-01-01,3985.0,8620.0,11704.0,16353.0,21115.0,23037.0,25060.0,25567.0,25847.0,26035.0,...,26339.8,26339.8,26339.8,26339.8,26339.8,26339.8,26339.8,NaN,NaN,NaN
1990-01-01,4310.0,8010.0,13330.0,18842.0,23155.0,24727.0,25889.0,26639.0,26659.0,27078.0,...,27373.0,27373.0,27373.0,27373.0,27373.0,27373.0,NaN,NaN,NaN,NaN
1991-01-01,5456.0,10347.0,15719.0,22377.0,26814.0,28057.0,30818.0,32924.0,34310.0,34821.0,...,35437.0,35437.0,35437.0,35437.0,35437.0,NaN,NaN,NaN,NaN,NaN
1992-01-01,6015.0,13057.0,20015.0,27176.0,33927.0,36524.0,38966.0,39588.0,41164.0,41530.0,...,41905.0,41905.0,41905.0,41905.0,NaN,NaN,NaN,NaN,NaN,NaN
1993-01-01,7725.0,15591.0,23082.0,31271.0,36053.0,42638.0,46182.0,46677.0,46918.0,48641.0,...,49909.0,49909.0,49909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994-01-01,8656.0,20221.0,31161.0,40553.0,46093.0,50227.0,54195.0,55815.0,56825.0,58423.0,...,60255.0,60255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# creat a new rocky object
r = rocky()

# inspect object
print(r)

rocky(id='rocky0', mod=(), f=(), plot=(), t=())


In [ ]:
## Could read in sample data (taylor_ashe, mack_1994) to try it out
r.SampleTri('taylor_ashe')
# r.paid_loss.reset_base_linear_model(var_name='accident_period', trends=False)

## Could copy a triangle from a file to the clipboard and then read it in
# r.FromClipboard('paid_loss')

## for this demo, we'll just read in a triangle from a link ratio file

# triangle file path for WC 4Q2022
# path = r'O:\STAFFHQ\SYMDATA\Actuarial\Reserving Applications\IBNR Allocation\4Q2022 Analysis\Workers Compensation 4Q2022.xlsb'

# read in triangle from file
# r.FromExcel(path, 1, 'paid_loss', 'Gross Paid Ultimates', "a28:u48") # takes filepath, number of origin columns, name of triangle, sheet name, and range of triangle
print(r)

rocky(id='rocky0', mod=(), f=(), plot=(), t="paid_loss")


In [ ]:
# access triangle object
r.paid_loss.tri

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001-01-01,357848.0,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
2002-01-01,352118.0,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2003-01-01,290507.0,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
2004-01-01,310608.0,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
2005-01-01,443160.0,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
2006-01-01,396132.0,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
2007-01-01,440832.0,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-01,359480.0,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-01,376686.0,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r.paid_loss.acc.dt.year

0    2001
1    2002
2    2003
3    2004
4    2005
5    2006
6    2007
7    2008
8    2009
9    2010
Name: accident_period, dtype: int32

In [ ]:
r.AddModel('glm', 'glm', 'paid_loss')
r

rocky(id='rocky0', mod="glm", f=(), plot=(), t="paid_loss")

In [ ]:
# access linear design matrix:
r.paid_loss.X_base.head(10)

,is_observed,intercept,accident_period_2002,accident_period_2003,accident_period_2004,accident_period_2005,accident_period_2006,accident_period_2007,accident_period_2008,accident_period_2009,accident_period_2010,development_period_002,development_period_003,development_period_004,development_period_005,development_period_006,development_period_007,development_period_008,development_period_009,development_period_010
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
# quick ATA factor summary
r.paid_loss.ata_summary()

Age-to-Age Factors as of (months),1,2,3,4,5,6,7,8,9
2001,3.143,1.543,1.278,1.238,1.209,1.044,1.04,1.063,1.018
2002,3.511,1.755,1.545,1.133,1.084,1.128,1.057,1.086,
2003,4.448,1.717,1.458,1.232,1.037,1.12,1.061,,
2004,4.568,1.547,1.712,1.073,1.087,1.047,,,
2005,2.564,1.873,1.362,1.174,1.138,,,,
2006,3.366,1.636,1.369,1.236,,,,,
2007,2.923,1.878,1.439,,,,,,
2008,3.953,2.016,,,,,,,
2009,3.619,,,,,,,,
Vol Wtd,,,,,,,,,


In [ ]:
# Triangle data type actually has a robust chain-ladder implementation that can be accessed

# this isn't important for right now, but it is there and it works well:
r.paid_loss.ult('vwa', 5, tail=1.061)

Accident Period
2001-01-01    4139452.0
2002-01-01    5765176.0
2003-01-01    5706935.0
2004-01-01    5621078.0
2005-01-01    5154550.0
2006-01-01    5422953.0
2007-01-01    5961413.0
2008-01-01    7197996.0
2009-01-01    6120629.0
2010-01-01    5011529.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [ ]:
# now add a GLM model to the triangle:
r.AddModel('icrfs', 'loglinear', 'paid_loss')
print(r)

This is for testing and compatability only. Please do not assign much credibility to these estimates for the purposes selecting carried reserves.
rocky(id='rocky0', mod=("glm", "icrfs"), f=(), plot=(), t="paid_loss")


In [ ]:
r.icrfs.GetWeights()[:5]

0    3.038427
1    3.038427
2    3.038427
3    3.038427
4    3.038427
Name: weights, dtype: float64

In [ ]:
r.icrfs.SetHyperparameters(alpha=0, l1_ratio=0.5)
r.icrfs

loglinear(alpha=0.0, l1_ratio=0.50)

In [ ]:
r.icrfs.GetY('train')[:5]

0    13.899203
1    13.883061
2    13.690723
3    13.757627
4    14.113026
dtype: float64

In [ ]:
r.icrfs.Fit()

In [ ]:
r.icrfs.GetY('train', log=True)[:5]

0    13.899203
1    13.883061
2    13.690723
3    13.757627
4    14.113026
dtype: float64

In [ ]:
r.icrfs.BasicHeteroAdjustment()[:5]

0    3.038427
1    3.038427
2    3.038427
3    3.038427
4    3.038427
Name: hetero_adjustment, dtype: float64

In [ ]:
r.icrfs.GetY('train', log=True)[:5]

0    13.899203
1    13.883061
2    13.690723
3    13.757627
4    14.113026
dtype: float64

In [ ]:
# r.icrfs.TuneFitHyperparameters()
r.icrfs.SetHyperparameters(alpha=0, l1_ratio=0.5)
r.icrfs

loglinear(alpha=0.0, l1_ratio=0.50)

In [ ]:
r.icrfs.Fit()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
r.icrfs.model.coef_

array([13.63117943,  0.36100181,  0.28223955,  0.17119397,  0.28222204,
        0.31174862,  0.39204854,  0.48026995,  0.34516321,  0.22859802,
        0.91118965,  0.02753002,  0.0262615 , -0.58177966, -0.38811074,
       -0.11316027, -0.32120766,  0.38576977, -1.33983425])

In [ ]:
# r.glm.TuneFitHyperparameters()
r.glm.SetHyperparameters(alpha=0.2, power=2.3)
r.glm

tweedieGLM(alpha=0.2, power=2.3)

In [ ]:
r.icrfs.GetY('train')[:5]

0    13.899203
1    13.883061
2    13.690723
3    13.757627
4    14.113026
dtype: float64

In [ ]:
r.icrfs

loglinear(alpha=0.0, l1_ratio=0.50)

In [ ]:
## now that we have a set of hyperparameters, we can reuse them
## next time using model.SetHyperparameters(alpha, power):
# -> r.tweedie.SetHyperparameters(alpha=0.4, power=1.8)

# fit base model to the data
r.icrfs.Fit()
r.glm.Fit()
print(r)

rocky(id='rocky0', mod=("glm", "icrfs"), f=(), plot=(), t="paid_loss")


In [ ]:
r.icrfs.GetYhat('train').shape

(55,)

In [ ]:
dat = r.icrfs.GetX("train")
DYwGp = r.icrfs.GetHeteroGp().loc[r.icrfs.GetIdx('train')]
DYwGp.rename(columns={DYwGp.columns[0]: "gp"}, inplace=True)
DYw = r.icrfs.GetWeights("train")
#this is a test


# print(f"dat: {dat.head(10)}")
# print(f"DYwGp: {DYwGp.head(10)}")

# join weight groups to data frame
# dat = dat.join(DYwGp)
# dat

In [ ]:
r.icrfs.GetWeights("train")[:5]

0    3.038427
1    3.038427
2    3.038427
3    3.038427
4    3.038427
Name: weights, dtype: float64

In [ ]:
resid2 = r.icrfs.RawResiduals().apply(lambda x: x**2)
resid2 

0     7.183685e-02
1     1.190713e-02
2     4.959360e-02
3     2.002262e-03
4     3.984994e-02
5     3.354093e-03
6     7.145486e-03
7     4.313752e-02
8     6.674782e-04
9     3.155444e-28
10    1.419305e-02
11    9.959037e-03
12    1.082414e-02
13    9.989994e-02
14    6.979736e-02
15    6.008402e-05
16    2.988650e-02
17    1.293570e-03
18    6.674782e-04
20    1.862049e-02
21    5.247969e-03
22    3.735878e-06
23    4.570020e-03
24    4.446213e-03
25    1.446346e-02
26    7.806920e-03
27    5.937116e-02
30    1.577465e-01
31    1.904009e-02
32    4.220720e-03
33    3.668954e-01
34    4.560881e-02
35    3.917466e-02
36    1.131344e-10
40    7.427429e-02
41    6.581309e-02
42    1.180852e-01
43    3.123773e-01
44    2.834982e-03
45    6.376843e-02
50    5.567360e-01
51    3.870988e-02
52    8.091111e-01
53    7.295321e-03
54    7.006302e-02
60    2.581336e-01
61    1.711546e-01
62    1.851627e-01
63    1.128592e-01
70    5.360242e-02
71    2.533483e-03
72    3.282918e-02
80    1.7587

In [ ]:
r.icrfs.GetDegreesOfFreedom()

36

In [ ]:
W_vec = r.icrfs.GetWeights(kind="train")

# get the diagonal matrix of weights
W = np.diag(W_vec.values.flatten())

# get the X matrix (design matrix)
X = r.icrfs.GetX(kind="train").values

# get the process variance
# if process_var_est == "ube":
V = r.icrfs._ProcessVarUBE()
# elif process_var_est == "mle":
#     V = r.icrfs._ProcessVarMLE()
# else:
#     raise ValueError("process_var_est must be 'ube' or 'mle'")

# # calculate the variance matrix
# varMatrix = (np.linalg.inv(W) - X @ np.linalg.inv(X.T @ W @ X) @ X.T) * V

# # return the standard errors
# se = np.sqrt(np.abs(np.diag(varMatrix)))
# return pd.Series(se, index=r.icrfs.GetX(kind="train").columns, name="Std Error")

In [ ]:
r.icrfs.RawResiduals().dropna()

0     2.680240e-01
1    -1.091198e-01
2    -2.226962e-01
3    -4.474664e-02
4     1.996245e-01
5     5.791453e-02
6     8.453098e-02
7    -2.076957e-01
8    -2.583560e-02
9     1.776357e-14
10    1.191346e-01
11   -9.979498e-02
12    1.040391e-01
13    3.160695e-01
14   -2.641919e-01
15    7.751388e-03
16   -1.728771e-01
17   -3.596624e-02
18    2.583560e-02
20   -1.364569e-01
21   -7.244287e-02
22   -1.932842e-03
23   -6.760192e-02
24    6.667993e-02
25   -1.202641e-01
26    8.835678e-02
27    2.436620e-01
30   -3.971731e-01
31    1.379858e-01
32    6.496707e-02
33    6.057189e-01
34   -2.135622e-01
35   -1.979259e-01
36   -1.063646e-05
40    2.725331e-01
41   -2.565406e-01
42    3.436352e-01
43   -5.589073e-01
44   -5.324455e-02
45    2.525241e-01
50    7.461474e-01
51   -1.967483e-01
52   -8.995060e-01
53    8.541265e-02
54    2.646942e-01
60   -5.080685e-01
61    4.137083e-01
62    4.303054e-01
63   -3.359453e-01
70   -2.315220e-01
71    5.033372e-02
72    1.811883e-01
80   -1.3261

In [ ]:
V

0.35311672580941256

In [ ]:
W

array([[3.03842661, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 3.03842661, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 3.03842661, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 3.03842661, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 3.03842661,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        3.03842661]])

In [ ]:
r.icrfs._StandardError()

ValueError: Length of values (55) does not match length of index (19)

In [ ]:
r.icrfs._CalcHeteroAdj()

LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [ ]:
para = r.icrfs.GetParameters().groupby('param_type')['param'].cumsum()
para

# model.feature_names_in_

0     1.950833
1     1.486848
2     2.603014
3     3.439503
4     4.058836
5     4.497487
6     4.784502
7     4.943370
8     4.999521
9     4.999521
10    1.486848
11    2.579745
12    3.367121
13    3.911446
14    4.279903
15    4.514840
16    4.641136
17    4.684375
18    4.684375
Name: param, dtype: float64

In [ ]:
r.icrfs._ProcessVarUBE().hist()

In [ ]:
r.icrfs.model

ElasticNet(alpha=3, fit_intercept=False, l1_ratio=0.05, max_iter=100000)

In [ ]:
r.icrfs.model.coef_

array([1.95083305, 1.48684759, 1.11616646, 0.83648878, 0.6193328 ,
       0.43865107, 0.28701556, 0.15886801, 0.056151  , 0.        ,
       1.48684788, 1.09289731, 0.78737558, 0.54432509, 0.36845754,
       0.23493658, 0.12629572, 0.04323888, 0.        ])

In [ ]:
r.icrfs.PredictTriangle().pivot_table(index='accident_period', columns='development_period', values='yhat').round(0)

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001,7.0,31.0,93.0,204.0,352.0,508.0,643.0,729.0,761.0,761.0
2002,31.0,138.0,411.0,902.0,1555.0,2248.0,2843.0,3225.0,3368.0,3368.0
2003,95.0,420.0,1253.0,2754.0,4747.0,6862.0,8679.0,9848.0,10283.0,10283.0
2004,219.0,970.0,2893.0,6358.0,10958.0,15839.0,20034.0,22731.0,23735.0,23735.0
2005,407.0,1802.0,5374.0,11811.0,20356.0,29424.0,37217.0,42227.0,44092.0,44092.0
2006,632.0,2794.0,8334.0,18314.0,31564.0,45626.0,57708.0,65477.0,68370.0,68370.0
2007,842.0,3723.0,11104.0,24403.0,42057.0,60794.0,76893.0,87245.0,91100.0,91100.0
2008,987.0,4364.0,13016.0,28605.0,49299.0,71261.0,90133.0,102267.0,106786.0,106786.0
2009,1044.0,4616.0,13768.0,30257.0,52146.0,75377.0,95339.0,108173.0,112953.0,112953.0


In [ ]:
r.paid_loss.incr_triangle.astype(float).round(0)

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001,357848.0,766940.0,610542.0,482940.0,527326.0,574398.0,146342.0,139950.0,227229.0,67948.0
2002,352118.0,884021.0,933894.0,1183289.0,445745.0,320996.0,527804.0,266172.0,425046.0,NaN
2003,290507.0,1001799.0,926219.0,1016654.0,750816.0,146923.0,495992.0,280405.0,NaN,NaN
2004,310608.0,1108250.0,776189.0,1562400.0,272482.0,352053.0,206286.0,NaN,NaN,NaN
2005,443160.0,693190.0,991983.0,769488.0,504851.0,470639.0,NaN,NaN,NaN,NaN
2006,396132.0,937085.0,847498.0,805037.0,705960.0,NaN,NaN,NaN,NaN,NaN
2007,440832.0,847631.0,1131398.0,1063269.0,NaN,NaN,NaN,NaN,NaN,NaN
2008,359480.0,1061648.0,1443370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,376686.0,986608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r.paid_loss.incr_triangle

development_period,1,2,3,4,5,6,7,8,9,10
accident_period,,,,,,,,,,
2001,357848.0,766940.0,610542.0,482940.0,527326.0,574398.0,146342.0,139950.0,227229.0,67948.0
2002,352118.0,884021.0,933894.0,1183289.0,445745.0,320996.0,527804.0,266172.0,425046.0,NaN
2003,290507.0,1001799.0,926219.0,1016654.0,750816.0,146923.0,495992.0,280405.0,NaN,NaN
2004,310608.0,1108250.0,776189.0,1562400.0,272482.0,352053.0,206286.0,NaN,NaN,NaN
2005,443160.0,693190.0,991983.0,769488.0,504851.0,470639.0,NaN,NaN,NaN,NaN
2006,396132.0,937085.0,847498.0,805037.0,705960.0,NaN,NaN,NaN,NaN,NaN
2007,440832.0,847631.0,1131398.0,1063269.0,NaN,NaN,NaN,NaN,NaN,NaN
2008,359480.0,1061648.0,1443370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,376686.0,986608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r.icrfs.hetero_gp

,development_period,hetero_001,hetero_002,hetero_003,hetero_004,hetero_005,hetero_006,hetero_007,hetero_008,hetero_009,hetero_010
0,1,True,False,False,False,False,False,False,False,False,False
1,1,True,False,False,False,False,False,False,False,False,False
2,1,True,False,False,False,False,False,False,False,False,False
3,1,True,False,False,False,False,False,False,False,False,False
4,1,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
95,10,False,False,False,False,False,False,False,False,False,True
96,10,False,False,False,False,False,False,False,False,False,True
97,10,False,False,False,False,False,False,False,False,False,True
98,10,False,False,False,False,False,False,False,False,False,True


In [ ]:
r.icrfs.GetHeteroGp()

# r.icrfs.(r.icrfs.GetX().loc[r.icrfs.forecast_index])

,development_period
0,1
1,1
2,1
3,1
4,1
...,...
95,10
96,10
97,10
98,10


In [ ]:
r.paid_loss.ult('vwa', 5, tail=1.061)

Accident Period
2001-01-01    4139452.0
2002-01-01    5765176.0
2003-01-01    5706935.0
2004-01-01    5621078.0
2005-01-01    5154550.0
2006-01-01    5422953.0
2007-01-01    5961413.0
2008-01-01    7197996.0
2009-01-01    6120629.0
2010-01-01    5011529.0
Name: Chain Ladder Ultimate Loss, dtype: float64

In [ ]:
r.icrfs.Ultimate()

Accident Period
2001-01-01    282773.0
2002-01-01    211386.0
2003-01-01    244726.0
2004-01-01    176377.0
2005-01-01    215390.0
2006-01-01    258108.0
2007-01-01    256830.0
2008-01-01    325206.0
2009-01-01    389263.0
2010-01-01    418850.0
Name: Log-linear Ultimate, dtype: float64

In [ ]:
r.glm.Ultimate()

Accident Period
2001-01-01    6922300.0
2002-01-01    5969505.0
2003-01-01    5691169.0
2004-01-01    6683660.0
2005-01-01    5488934.0
2006-01-01    6131080.0
2007-01-01    7080337.0
2008-01-01    5750500.0
2009-01-01    6336181.0
2010-01-01    5896610.0
Name: tweedieGLM Ultimate, dtype: float64

In [ ]:
# chain ladder ult vs GLM
df = pd.concat([pd.DataFrame(r.paid_loss.ult()), pd.DataFrame(r.icrfs.Ultimate()), pd.DataFrame(r.glm.Ultimate())], axis=1)
df = df.assign(Diff=df['Log-linear Ultimate'] - df['Chain Ladder Ultimate Loss']).astype(int)
df['Diagonal'] = r.paid_loss.diag().astype(int)
df['Unpaid'] = df['Log-linear Ultimate'] - df.Diagonal
df

,Chain Ladder Ultimate Loss,Log-linear Ultimate,tweedieGLM Ultimate,Diff,Diagonal,Unpaid
Accident Period,,,,,,
2001-01-01,3901463,282773,6922300,-3618690,3901463,-3618690
2002-01-01,5433719,211386,5969505,-5222333,5339085,-5127699
2003-01-01,5378826,244726,5691169,-5134100,4909315,-4664589
2004-01-01,5297906,176377,6683660,-5121529,4588268,-4411891
2005-01-01,4858200,215390,5488934,-4642810,3873311,-3657921
2006-01-01,5111171,258108,6131080,-4853063,3691712,-3433604
2007-01-01,5660771,256830,7080337,-5403941,3483130,-3226300
2008-01-01,6784799,325206,5750500,-6459593,2864498,-2539292
2009-01-01,5642266,389263,6336181,-5253003,1363294,-974031


In [ ]:
r.icrfs.plot.obs_pred(log=True, color='dev')

In [ ]:
r.tweedie.plot.residual('acc', log=True)

AttributeError: 'rocky' object has no attribute 'tweedie'

In [ ]:
r.tweedie.plot.residual('dev', log=False)

: 

In [ ]:
r.paid_loss.tri

: 

In [ ]:
## accident period parameters should roughly follow the first column:
r.paid_loss.tri.iloc[:, 0].plot()

: 

In [ ]:
# but the actual fitted parameters include regularization terms that smooth out some of these bumps
r.tweedie.PlotParameter('acc')

: 

In [ ]:
# does not include dev_period_0012 - first development period is assumed to be included in the first accident period parameter
r.tweedie.PlotParameter('dev')

: 

In [ ]:
r.tweedie.Ultimate()

: 

In [ ]:
# final 2 accident years don't change much from 2020 - can set parameters for these years to be 0 (for no change)
r.tweedie.ManualFit(
  accident_period_2021=0, accident_period_2022=0,
  accident_period_2005=0,
  accident_period_2013=0,
  accident_period_2016=0,
  accident_period_2018=0
  )

: 

In [ ]:
r.tweedie.PlotParameter('acc')

: 

In [ ]:
r.tweedie.Ultimate()

: 

In [ ]:
## TODO
# - add a method to do a future calendar year projection
# - add a method to combine parameters into a single parameter for each year

: 

In [ ]:
# add a second glm model that includes calendar year effects
r.AddModel('tweediecal', 'glm', 'paid_loss', cal=True)
print(r)

: 

In [ ]:
# can look at design matrix for this model -- it has 39 more columns for the 39 calendar periods in a 20x20 triangle
r.tweediecal.GetX()

: 

In [ ]:
# using same hyperparameters as before, but could decide to refit
r.tweediecal.SetHyperparameters(alpha=0.3, power=1.8)
r.tweediecal

: 

In [ ]:
r.tweediecal.Fit()

: 

In [ ]:
r.tweediecal.Ultimate()

: 

In [ ]:
r.tweediecal.PlotParameter('acc')

: 

In [ ]:
r.tweediecal.PlotParameter('dev')

: 

In [ ]:
r.tweediecal.PlotParameter('cal')

: 

In [ ]:
## add a 1% future cal trend (eg assume that trend reverses itself)
future_cy_trend = 0.01
manfit = {}
for i in range(21, 40):
  manfit[f"calendar_period_00{i}"] = future_cy_trend

r.tweediecal.ManualFit(**manfit)

: 

In [ ]:
r.tweediecal.PlotParameter('cal')

: 

In [ ]:
r.tweediecal.Ultimate()

: 

In [ ]:
def parse_param_name(x):
  if x.find("_") == -1:
    return x
  else:
    end = x.split("_")[len(x.split("_"))-1]
    out = x.replace(f"_{end}", "")[:3]
    return out
  

df = pd.DataFrame(dict(nm=r.tweediecal.GetParameterNames(), val=r.tweediecal.model.coef_))
parse_param_name(df.nm[2])

: 

In [ ]:
df = pd.DataFrame({
  'chain ladder': r.paid_loss.ult(tail=1.05),
  'tweedie': r.tweedie.Ultimate(tail=1.05),
  'tweediecal': r.tweediecal.Ultimate(tail=1.05)
}).round(-3)/1000

df

: 

In [ ]:
r.Add

: 